In [2]:
import torch
import transformers
import json
import typing
import pickle as pkl
import gc

import attack_utility
import experiment_logger
import random

In [3]:
MODEL_PATH = "/data/models/hf/Llama-3.2-1B-Instruct"
model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype=torch.float16)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH)
model.generation_config.pad_token_id = tokenizer.pad_token_id


In [32]:
model_comp = model.model
lm_head = model.lm_head

embedding = model_comp.embed_tokens
layers = model_comp.layers

embed_matrix = embedding.weight.data

In [33]:
NUM_SAMPLES = 1000
NUM_PAIRS = 1000

context_tokens = random.sample(list(range(len(tokenizer))), NUM_SAMPLES)
control_tokens = [(random.randint(0, len(tokenizer)), random.randint(0, len(tokenizer))) for _ in range(NUM_PAIRS)]

In [34]:
embed_matrix.shape

torch.Size([128256, 2048])

In [ ]:
def get_distances(index_to_flip: int, weight_matrix: torch.tensor):
    context_perm = torch.randperm(embed_matrix.shape[0])
    p_row = embed_matrix[context_perm[index_to_flip]]
    p_plus_k_rows = embed_matrix[context_perm[index_to_flip + 1:]]
    for x1_token, x2_token in control_tokens:
        x1_tensor = embed_matrix.clone()
        x1_tensor[index_to_flip] = 

In [28]:
t = torch.tensor([1, 2, 3])

In [30]:
t[[2, 1]]

tensor([3, 2])